<a href="https://colab.research.google.com/github/hazelerate/food_sales_prediction/blob/main/Project_1_Final_(Core).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data Cleaning

In [1]:
#mount data
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#import libraries
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [3]:
#load data
filename = '/content/sales_predictions.csv'
sales = pd.read_csv(filename)
sales.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [4]:
#run sales information to check columns, rows, etc.
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [5]:
#check if there are duplicates under Item_Identifier column
sales['Item_Identifier'].duplicated().sum()

6964

In [6]:
#drop duplicates
new_sales = sales.drop_duplicates(subset=['Item_Identifier'],keep='first')
new_sales.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [7]:
#Identify missing data
new_sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1559 entries, 0 to 7298
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            1559 non-null   object 
 1   Item_Weight                1305 non-null   float64
 2   Item_Fat_Content           1559 non-null   object 
 3   Item_Visibility            1559 non-null   float64
 4   Item_Type                  1559 non-null   object 
 5   Item_MRP                   1559 non-null   float64
 6   Outlet_Identifier          1559 non-null   object 
 7   Outlet_Establishment_Year  1559 non-null   int64  
 8   Outlet_Size                1103 non-null   object 
 9   Outlet_Location_Type       1559 non-null   object 
 10  Outlet_Type                1559 non-null   object 
 11  Item_Outlet_Sales          1559 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 158.3+ KB


In [8]:
#check for missing values
print(new_sales.isna().sum().sum(), 'missing values')

710 missing values


In [9]:
#train test split
X = new_sales.drop(columns=['Item_MRP','Outlet_Identifier','Outlet_Establishment_Year','Outlet_Location_Type','Item_Outlet_Sales'])
y = new_sales['Item_Outlet_Sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [10]:
#Select Columns
num_selector = make_column_selector(dtype_include='number')
cat_selector = make_column_selector(dtype_include='object')

In [11]:
#instantiate SimpleImputer to fill missing data with the most frequent value
mean_imputer = SimpleImputer(strategy='mean')
freq_imputer = SimpleImputer(strategy='most_frequent')
scaler = StandardScaler()
ohe_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

In [12]:
#combine to a pipeline
num_pipe = make_pipeline(mean_imputer, scaler)
cat_pipe = make_pipeline(freq_imputer, ohe_encoder)

In [13]:
#match columns
num_tuple = (num_pipe, num_selector)
cat_tuple = (cat_pipe, cat_selector)
column_transformer = make_column_transformer(num_tuple, cat_tuple)

In [14]:
column_transformer.fit(X_train)

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fa640cd76d0>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fa640cd7dd0>

In [15]:
X_train_processed = column_transformer.transform(X_train)
X_test_processed = column_transformer.transform(X_test)

In [16]:
reg = LinearRegression()

In [17]:
reg.fit(X_train_processed,y_train)

LinearRegression()

In [18]:
train_score = reg.score(X_train_processed, y_train)
print(train_score)

1.0


In [19]:
test_score = reg.score(X_test_processed, y_test)
print(test_score)

0.20018540868903534


In [20]:
model = LinearRegression()

In [21]:
model.fit(X_train_processed,y_train)

LinearRegression()

In [22]:
# make prediction for test set
prediction = model.predict(X_test_processed)

In [23]:
# R2 score
r2_score(y_test, prediction)

0.20018540868903534

In [24]:
# R2 score
r2_score(y_test, prediction)

0.20018540868903534

In [25]:
# create the model
from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor(random_state = 42)

In [26]:
# train the model
model.fit(X_train_processed, y_train)

DecisionTreeRegressor(random_state=42)

In [27]:
# evaluate the model
train_score = model.score(X_train_processed, y_train)
test_score = model.score(X_test_processed, y_test)
print(train_score)
print(test_score)

1.0
-0.09989538772643902


In [28]:
# make prediction for test set
prediction = model.predict(X_test_processed)

In [29]:
# R2 score
r2_score(y_test, prediction)

-0.09989538772643902

In [30]:
# RMSE
np.sqrt(mean_squared_error(y_test, prediction))

1957.8865386946818